### ***FAKE NEWS PREDICTION***

***1: Fake News***
<br>***0: Real News***

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
# from nltk.stem.porter import PorterStemmer (not a good stemmer ig)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [45]:
nltk.download('Stopwords')
nltk.download('wordnet')

[nltk_data] Error loading Stopwords: Package 'Stopwords' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arrma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [46]:
#printing the stopwords in english:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

***data pre-Processing:***

In [47]:
df = pd.read_csv('4train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [49]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [50]:
#replacing the null values with ''empty string''
df = df.fillna('')

In [51]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [52]:
#merging the auther name and news title:
df['content'] = df['author']+' - '+df['title']

In [53]:
df['content']

0        Darrell Lucus - House Dem Aide: We Didn’t Even...
1        Daniel J. Flynn - FLYNN: Hillary Clinton, Big ...
2        Consortiumnews.com - Why the Truth Might Get Y...
3        Jessica Purkiss - 15 Civilians Killed In Singl...
4        Howard Portnoy - Iranian woman jailed for fict...
                               ...                        
20795    Jerome Hudson - Rapper T.I.: Trump a ’Poster C...
20796    Benjamin Hoffman - N.F.L. Playoffs: Schedule, ...
20797    Michael J. de la Merced and Rachel Abrams - Ma...
20798    Alex Ansary - NATO, Russia To Hold Parallel Ex...
20799            David Swanson - What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [54]:
#separating the data & label:
X = df.drop(columns='label', axis=1)
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus - House Dem Aide: We Didn’t Even...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn - FLYNN: Hillary Clinton, Big ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com - Why the Truth Might Get Y...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss - 15 Civilians Killed In Singl...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy - Iranian woman jailed for fict...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson - Rapper T.I.: Trump a ’Poster C...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman - N.F.L. Playoffs: Schedule, ..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams - Ma...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary - NATO, Russia To Hold Parallel Ex..."


In [55]:
Y = df['label']
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


***Stemming:***
* *stemming is the process of reducing a word to its Root word.*
* *example:*
*`actor, actress, acting` --> act*

In [56]:
# from nltk.stem.porter import PorterStemmer

In [57]:
# port_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [77]:
# def stemming(content):
#     stemmed_content = re.sub('[^a-zA-Z]',' ', content) #Removes Non-Alphabetic Characters
#     stemmed_content = stemmed_content.lower() #Converts Text to Lowercase
#     stemmed_content = stemmed_content.split() #Tokenizes the Text
#     stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #Removes Stopwords (Common Words)
#     stemmed_content = ' '.join(stemmed_content)
#     return stemmed_content

def preprocess_text(text):
    text = str(text).lower() #lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text) #rmeove the punctuations
    words = text.split() #tokenization
    words = [word for word in words if word not in stop_words] #remove Stopwords
    words = [lemmatizer.lemmatize(word) for word in words] #lemmatize

    return " ".join(words)

In [78]:
# df['content'] = df['content'].apply(stemming)
df['content'] = df['content'].apply(preprocess_text)

In [79]:
df['content']

0        darrell lucus house dem aide didnt even see co...
1        daniel j flynn flynn hillary clinton big woman...
2                  consortiumnewscom truth might get fired
3        jessica purkiss civilian killed single u airst...
4        howard portnoy iranian woman jailed fictional ...
                               ...                        
20795    jerome hudson rapper ti trump poster child whi...
20796    benjamin hoffman nfl playoff schedule matchup ...
20797    michael j de la merced rachel abrams macys sai...
20798    alex ansary nato russia hold parallel exercise...
20799                           david swanson keep f alive
Name: content, Length: 20800, dtype: object

In [80]:
#separating the data and label
X = df['content'].values
X

array(['darrell lucus house dem aide didnt even see comeys letter jason chaffetz tweeted',
       'daniel j flynn flynn hillary clinton big woman campus breitbart',
       'consortiumnewscom truth might get fired', ...,
       'michael j de la merced rachel abrams macys said receive takeover approach hudson bay new york time',
       'alex ansary nato russia hold parallel exercise balkan',
       'david swanson keep f alive'], dtype=object)

In [81]:
X.shape

(20800,)

In [82]:
y = df['label'].values
y

array([1, 0, 1, ..., 0, 1, 1])

In [83]:
y.shape

(20800,)

In [84]:
#converting the textual data to numerical data:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 208432 stored elements and shape (20800, 22566)>

***Splitting the dataset into training & test data:***

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=2)

***Training The Model:***

In [86]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='saga')  # 'saga' supports sparse matrices
model.fit(X_train, y_train)

LogisticRegression(solver='saga')

***model Evolution:***

In [87]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [88]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9870192307692308


In [89]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [90]:
print('Accuracy score of the testing data : ', testing_data_accuracy)

Accuracy score of the testing data :  0.9771634615384616


***making a Predictive system:***

In [93]:
X_new = X_test[2222]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
    print('The news is Real')
else:
    print('The news is Fake')

[1]
The news is Fake


In [94]:
print(y_test[2222])

1
